# STA 141B Data & Web Technologies for Data Analysis

### Lecture 12, 11/21/23, SQL

### Announcements

 - No lecture on Thursday!
 - Exam is graded and online, if yours is not shown, please alert me on Piazza! 

### Last week's topics

- NLP: LDA
- Selenium

### Today's topics
- Databases
- Structured Query Language
 
### Datasets

* The Suppliers Database

### References

* [W3 Schools SQL Tutorial](https://www.w3schools.com/sql/)
* [SQL Cheatsheet](https://www.sqltutorial.org/sql-cheat-sheet/)

[PDSH]: https://jakevdp.github.io/PythonDataScienceHandbook/
[ProGit]: https://git-scm.com/book/
[nlpp]: https://www.nltk.org/book/
[atap]: https://search.library.ucdavis.edu/primo-explore/fulldisplay?docid=01UCD_ALMA51320822340003126&context=L&vid=01UCD_V1&search_scope=everything_scope&tab=default_tab&lang=en_US

## Databases

A _database_ is a collection of data. There are several different models for how to organize data in a database; these are called _database models_. In this context, "model" refers to a design or mental model, not a statistical model.

The _relational model_ organizes data as a collection of tables. Tables have rows (also called _tuples_ or _records_) and columns (also called _attributes_). Most tables have a _key_ column that is unique for each row and _relates_ the table to other tables. The relational model is the most popular database model by far, and the one we'll focus on in this course.

There are also many different software programs for managing databases, called _database management systems_ (DBMS). Each DBMS usually has its own format for storing data on disk, independent of the database model. Some popular DBMSes are:

* [SQLite](https://www.sqlite.org/)
* [MySQL](https://www.mysql.com/)
* [Microsoft SQL Server](https://www.microsoft.com/en-us/sql-server)
* [PostgreSQL](https://www.postgresql.org/)

Why use a database? There are several reasons:

* Your data may already be in a database, so converting to another format is extra work.
* Database operations are highly optimized, so they typically take less time and memory than an equivalent operation in Python.
* Database operations can run on datasets that are too large to fit in memory. Doing this in Python requires special programming strategies.
* Many DBMSes provide built-in version control, multi-user access, and security checks.
* Databases can be updated in real time.

## Structured Query Language

_Structured query language_ (SQL) is a language designed for querying information in relational databases.

A free SQL tutorial is available [here](https://www.w3schools.com/sql/).

### Getting Connected

There are several ways to connect to a database and run SQL queries from Python:

* The built-in __sqlite3__ module, which only supports SQLite.
* The __sqlalchemy__ package, a unified interface for a variety of different SQL database formats (more than just SQLite). See the [tutorial](http://docs.sqlalchemy.org/en/latest/core/tutorial.html) for more details.

We'll use a SQLite database here, since SQLite is possibly [the most-used database engine in the world](https://sqlite.org/mostdeployed.html). SQLite's popularity is partly due to its reliability, easy setup, and broad range of features.

Let's connect to the suppliers database:

In [1]:
import numpy as np
import pandas as pd

In [2]:
import sqlite3 as sql

To connect to a database, use the module's `connect()` function. This is similar to opening a file; you should close the database when you're done using it.

In [3]:
db = sql.connect("../data/suppliers.sqlite")

In [4]:
db

To execute a SQL query, use the connection's `.execute()` method. This returns a _cursor_, which is a pointer to the results in the database (imagine a finger pointing at the results).

SQLite databases store metadata in a special table called `sqlite_master`. We can use `sqlite_master` to find out the names of the other tables in the database.

In [5]:
cur = db.execute("SELECT * FROM sqlite_master")

In [6]:
type(cur)

sqlite3.Cursor

To get the results from the database, use one of the cursor's fetch methods. The `.fetchall()` method returns all rows in the result.

In [7]:
cur.fetchall()

[('table',
  'Suppliers',
  'Suppliers',
  2,
  'CREATE TABLE Suppliers (\n  SupplierID integer,\n  SupplierName text,\n  Status integer,\n  City text,\n  PRIMARY KEY(SupplierID)\n)'),
 ('table',
  'Parts',
  'Parts',
  3,
  'CREATE TABLE Parts (\n  PartID integer,\n  PartName text,\n  Color text,\n  Weight real,\n  City text,\n  PRIMARY KEY(PartID)\n)'),
 ('table',
  'SupplierParts',
  'SupplierParts',
  4,
  'CREATE TABLE SupplierParts (\n  PartID integer,\n  SupplierID integer,\n  Qty integer,\n  PRIMARY KEY(PartID, SupplierID)\n)'),
 ('index', 'sqlite_autoindex_SupplierParts_1', 'SupplierParts', 5, None)]

By default, `sqlite3` will return rows as tuples. If you'd rather have the rows as dictionaries indexed by column name, set the `.row_factory` attribute on the database connection.

In [8]:
db.row_factory = sql.Row

Now the rows will behave like dictionaries:

In [9]:
cur = db.execute("SELECT * FROM sqlite_master")
rows = cur.fetchall()
dict(rows[0])

{'type': 'table',
 'name': 'Suppliers',
 'tbl_name': 'Suppliers',
 'rootpage': 2,
 'sql': 'CREATE TABLE Suppliers (\n  SupplierID integer,\n  SupplierName text,\n  Status integer,\n  City text,\n  PRIMARY KEY(SupplierID)\n)'}

In [10]:
rows

Don't forget to close the database when you're done!

In [11]:
# db.close()

We'll generally use the `pd.read_sql()` function in __pandas__ to run our SQL queries. 

The function takes a SQL query and an open database connection as arguments, so you still need to connect to the database first with `sqlite3` or `sqlalchemy`. The result of the query is returned as a data frame.

### `SELECT`

The `SELECT` command selects rows from a table. Most of your SQL queries will start with `SELECT`. The syntax is:

```sql
SELECT col1, col2, ... FROM my_table;
```

Here `col1`, `col2`, and so on are column names and `my_table` is a table name. You can select all columns with an asterisk  `*`.

SQL is not case-sensitive and ignores whitespace, but the convention is to write SQL keywords in uppercase and column/table names in lowercase. A semicolon `;` marks the end of a SQL query, but this is optional for many tools.

In [13]:
# db.excute("SELECT * FROM file_name")

In [14]:
pd.read_sql("SELECT * FROM sqlite_master", db) # ; is optional but is preferred

,type,name,tbl_name,rootpage,sql
0,table,Suppliers,Suppliers,2,CREATE TABLE Suppliers (\n SupplierID integer...
1,table,Parts,Parts,3,"CREATE TABLE Parts (\n PartID integer,\n Par..."
2,table,SupplierParts,SupplierParts,4,CREATE TABLE SupplierParts (\n PartID integer...
3,index,sqlite_autoindex_SupplierParts_1,SupplierParts,5,None


In [15]:
# select only name and type from the data
pd.read_sql("SELECT name, type FROM sqlite_master", db)

,name,type
0,Suppliers,table
1,Parts,table
2,SupplierParts,table
3,sqlite_autoindex_SupplierParts_1,index


In [16]:
pd.read_sql("SELECT * FROM suppliers;", db)

,SupplierID,SupplierName,Status,City
0,1,Smith,20,London
1,2,Jones,10,Paris
2,3,Blake,30,Paris
3,4,Clark,20,London
4,5,Adams,30,Athens


In [17]:
pd.read_sql("SELECT * FROM parts;", db)

,PartID,PartName,Color,Weight,City
0,1,Nut,Red,12.0,London
1,2,Bolt,Green,17.0,Paris
2,3,Screw,Blue,17.0,Oslo
3,4,Screw,Red,14.0,London
4,5,Cam,Blue,12.0,Paris
5,6,Cog,Red,19.0,London


In [18]:
pd.read_sql("SELECT * FROM supplierparts;", db)

,PartID,SupplierID,Qty
0,1,1,300
1,1,2,200
2,1,3,400
3,1,4,200
4,1,5,100
5,1,6,100
6,2,1,300
7,2,2,400
8,3,2,200
9,4,2,200


### `LIMIT`

The `SELECT` command can be extended with many other keywords.

The first of these is `LIMIT`, which limits the number of rows returned. `LIMIT` is the SQL equivalent of Pandas' `.head()` method.

In [19]:
pd.read_sql("SELECT * FROM supplierparts LIMIT 3", db)

,PartID,SupplierID,Qty
0,1,1,300
1,1,2,200
2,1,3,400


### `DISTINCT`

The `DISTINCT` keyword limits rows to distinct results. `DISTINCT` is the SQL equivalent of Pandas' `.drop_duplicates()` method.

Keep in mind that `DISTINCT` applies to all of the selected columns, not just one column.

In [20]:
pd.read_sql("SELECT color, city FROM parts", db)

,Color,City
0,Red,London
1,Green,Paris
2,Blue,Oslo
3,Red,London
4,Blue,Paris
5,Red,London


In [21]:
pd.read_sql("SELECT DISTINCT color, city FROM parts;", db)

,Color,City
0,Red,London
1,Green,Paris
2,Blue,Oslo
3,Blue,Paris


### `ORDER BY`

The `ORDER BY` keyword sorts the returned rows. `ORDER BY` is the SQL equivalent of Pandas' `.sort_values()` method.

In [23]:
pd.read_sql("SELECT * FROM parts ORDER BY partid LIMIT 5;", db)

,PartID,PartName,Color,Weight,City
0,1,Nut,Red,12.0,London
1,2,Bolt,Green,17.0,Paris
2,3,Screw,Blue,17.0,Oslo
3,4,Screw,Red,14.0,London
4,5,Cam,Blue,12.0,Paris


Add the suffix `ASC` for an ascending sort (smallest to largest) and `DESC` for a descending sort (largest to smallest).

In SQLite, the default is ascending, but other other databases may differ.

In [24]:
pd.read_sql("SELECT * FROM parts ORDER BY weight DESC;", db)

,PartID,PartName,Color,Weight,City
0,6,Cog,Red,19.0,London
1,2,Bolt,Green,17.0,Paris
2,3,Screw,Blue,17.0,Oslo
3,4,Screw,Red,14.0,London
4,1,Nut,Red,12.0,London
5,5,Cam,Blue,12.0,Paris


In [25]:
pd.read_sql("SELECT LOG(partid) FROM parts ORDER BY weight DESC, city;", db)

,LOG(partid)
0,0.778151
1,0.477121
2,0.301030
3,0.602060
4,0.000000
5,0.698970


### `WHERE`

`WHERE` puts conditions on the rows returned. `WHERE` is the SQL equivalent of subsetting.

You can use `=` to test equality. Other comparison operators, such as `>=`, are also available.

In [26]:
pd.read_sql("SELECT * FROM parts", db)

,PartID,PartName,Color,Weight,City
0,1,Nut,Red,12.0,London
1,2,Bolt,Green,17.0,Paris
2,3,Screw,Blue,17.0,Oslo
3,4,Screw,Red,14.0,London
4,5,Cam,Blue,12.0,Paris
5,6,Cog,Red,19.0,London


In [27]:
pd.read_sql("SELECT * FROM parts WHERE city = 'London' OR color = 'Green';", db)

,PartID,PartName,Color,Weight,City
0,1,Nut,Red,12.0,London
1,2,Bolt,Green,17.0,Paris
2,4,Screw,Red,14.0,London
3,6,Cog,Red,19.0,London


You can use `AND` and `OR` to combine conditions. You can also use parenthesis to indicate the order of operations.

In [28]:
pd.read_sql("SELECT * FROM parts WHERE city = 'London' AND color = 'Green';", db)

,PartID,PartName,Color,Weight,City


You can use `IN` to check whether a value is in a collection of values.

In [29]:
pd.read_sql("SELECT * FROM parts WHERE city IN ('Paris', 'London');", db)

,PartID,PartName,Color,Weight,City
0,1,Nut,Red,12.0,London
1,2,Bolt,Green,17.0,Paris
2,4,Screw,Red,14.0,London
3,5,Cam,Blue,12.0,Paris
4,6,Cog,Red,19.0,London


In [30]:
pd.read_sql("SELECT * FROM parts;", db)

,PartID,PartName,Color,Weight,City
0,1,Nut,Red,12.0,London
1,2,Bolt,Green,17.0,Paris
2,3,Screw,Blue,17.0,Oslo
3,4,Screw,Red,14.0,London
4,5,Cam,Blue,12.0,Paris
5,6,Cog,Red,19.0,London


SQL's `LIKE` keyword does simple pattern-matching language for strings. This is less powerful than regular expressions, but still useful.

* `%` matches zero or more of any character, similar to regex (re) `.*`
* `_` matches any one character, similar to regex `.`

In other databases (but not SQLite):
* `[]` matches any one of the characters you put inside the brackets, identical to regex `[]`

In [31]:
pd.read_sql("SELECT * FROM parts WHERE city LIKE '%o%';", db) #ends with s

,PartID,PartName,Color,Weight,City
0,1,Nut,Red,12.0,London
1,3,Screw,Blue,17.0,Oslo
2,4,Screw,Red,14.0,London
3,6,Cog,Red,19.0,London


The `BETWEEN` ... `AND` keyword is useful for selecting ranges.

In [32]:
pd.read_sql("SELECT * FROM parts WHERE weight BETWEEN 14 AND 19;", db)

,PartID,PartName,Color,Weight,City
0,2,Bolt,Green,17.0,Paris
1,3,Screw,Blue,17.0,Oslo
2,4,Screw,Red,14.0,London
3,6,Cog,Red,19.0,London


### Operators

You can use arithmetic operators `+`, `-`, `*`, `\`, `%` on SQL columns to perform columnwise computations. These are the SQL equivalent of vectorized arithmetic.

In [33]:
pd.read_sql("SELECT weight * weight, * FROM parts", db)

# the first *: multiplication, and the second *: reads all columns.

,weight * weight,PartID,PartName,Color,Weight,City
0,144.0,1,Nut,Red,12.0,London
1,289.0,2,Bolt,Green,17.0,Paris
2,289.0,3,Screw,Blue,17.0,Oslo
3,196.0,4,Screw,Red,14.0,London
4,144.0,5,Cam,Blue,12.0,Paris
5,361.0,6,Cog,Red,19.0,London


In [34]:
pd.read_sql("SELECT weight + weight AS double_weight, * FROM parts", db)

,double_weight,PartID,PartName,Color,Weight,City
0,24.0,1,Nut,Red,12.0,London
1,34.0,2,Bolt,Green,17.0,Paris
2,34.0,3,Screw,Blue,17.0,Oslo
3,28.0,4,Screw,Red,14.0,London
4,24.0,5,Cam,Blue,12.0,Paris
5,38.0,6,Cog,Red,19.0,London


In [35]:
pd.read_sql('''
    SELECT weight + weight, 
    * FROM parts WHERE weight * weight > 200;
''', db)

,weight + weight,PartID,PartName,Color,Weight,City
0,34.0,2,Bolt,Green,17.0,Paris
1,34.0,3,Screw,Blue,17.0,Oslo
2,38.0,6,Cog,Red,19.0,London


### `AS`

You can rename a column with the `AS` keyword. This keyword is especially useful together with SQL arithmetic operators and functions.

In [36]:
pd.read_sql('''
    SELECT weight + weight AS weightsum, 
    * FROM parts WHERE weight * weight > 200;
''', db)

,weightsum,PartID,PartName,Color,Weight,City
0,34.0,2,Bolt,Green,17.0,Paris
1,34.0,3,Screw,Blue,17.0,Oslo
2,38.0,6,Cog,Red,19.0,London


### Functions & Aggregation

SQL has built-in functions, which vary from one DBMS to another. The SQL cheatsheet lists most of the functions supported by SQLite.

Most SQL functions aggregate data in a column, summarizing that column somehow.

In [37]:
pd.read_sql('''
    SELECT weight * 12 AS multiplied_weight, 
    * FROM parts;
''', db)

,multiplied_weight,PartID,PartName,Color,Weight,City
0,144.0,1,Nut,Red,12.0,London
1,204.0,2,Bolt,Green,17.0,Paris
2,204.0,3,Screw,Blue,17.0,Oslo
3,168.0,4,Screw,Red,14.0,London
4,144.0,5,Cam,Blue,12.0,Paris
5,228.0,6,Cog,Red,19.0,London


In [39]:
pd.read_sql("SELECT AVG(weight * 12) AS avg_multiplied_weight FROM parts;", db)

,avg_multiplied_weight
0,182.0


In [40]:
pd.read_sql("SELECT COUNT(*) AS countcolumn FROM parts;", db)

,countcolumn
0,6


In [41]:
pd.read_sql("SELECT upper(city) AS uppercity, * FROM parts;", db)

,uppercity,PartID,PartName,Color,Weight,City
0,LONDON,1,Nut,Red,12.0,London
1,PARIS,2,Bolt,Green,17.0,Paris
2,OSLO,3,Screw,Blue,17.0,Oslo
3,LONDON,4,Screw,Red,14.0,London
4,PARIS,5,Cam,Blue,12.0,Paris
5,LONDON,6,Cog,Red,19.0,London


### `GROUP BY`

The `GROUP BY` keyword groups rows before they are aggregated. `GROUP BY` is the SQL equivalent of Pandas' `.groupby()` method.

In [42]:
pd.read_sql("SELECT AVG(weight) FROM parts;", db)

,AVG(weight)
0,15.166667


In [43]:
pd.read_sql("SELECT AVG(weight), city FROM parts GROUP BY city;", db)

,AVG(weight),City
0,15.0,London
1,17.0,Oslo
2,14.5,Paris


In [44]:
pd.read_sql("SELECT AVG(weight), city, partname FROM parts GROUP BY city, partname;", db)

,AVG(weight),City,PartName
0,19.0,London,Cog
1,12.0,London,Nut
2,14.0,London,Screw
3,17.0,Oslo,Screw
4,17.0,Paris,Bolt
5,12.0,Paris,Cam


### `HAVING`

The `WHERE` keyword puts conditions on the rows returned _before computing any aggregate functions._ So use `WHERE` to remove rows before aggregation.

The `HAVING` keyword puts conditions on the rows returned _after computing any aggregate functions._ So use `HAVING` to remove rows after aggregation. `HAVING` is most useful for conditions on the result of an aggregate.

In [45]:
# It's an error to use WHERE to put a condition on an aggregated column.
pd.read_sql('''
    SELECT AVG(weight) AS avg_weight, city 
    FROM parts WHERE weight > 15 GROUP BY city HAVING avg_weight = 17 OR avg_weight = 19
''', db)

,avg_weight,City
0,19.0,London
1,17.0,Oslo
2,17.0,Paris


In [46]:
pd.read_sql("SELECT AVG(weight) AS avg_weight, city FROM parts GROUP BY city HAVING avg_weight > 15", db)

,avg_weight,City
0,17.0,Oslo


### Joins

A `JOIN` combines two tables using a column they have in common. `JOIN` is the SQL equivalent of Pandas' `.join()` method and `pd.merge()` function.

In [47]:
pd.read_sql("SELECT * FROM supplierparts", db)

,PartID,SupplierID,Qty
0,1,1,300
1,1,2,200
2,1,3,400
3,1,4,200
4,1,5,100
5,1,6,100
6,2,1,300
7,2,2,400
8,3,2,200
9,4,2,200


In [48]:
pd.read_sql("""
    SELECT * FROM
    supplierparts
    LEFT JOIN
    parts
    ON supplierparts.partid = parts.partid
""", db)

,PartID,SupplierID,Qty,PartID,PartName,Color,Weight,City
0,1,1,300,1,Nut,Red,12.0,London
1,1,2,200,1,Nut,Red,12.0,London
2,1,3,400,1,Nut,Red,12.0,London
3,1,4,200,1,Nut,Red,12.0,London
4,1,5,100,1,Nut,Red,12.0,London
5,1,6,100,1,Nut,Red,12.0,London
6,2,1,300,2,Bolt,Green,17.0,Paris
7,2,2,400,2,Bolt,Green,17.0,Paris
8,3,2,200,3,Screw,Blue,17.0,Oslo
9,4,2,200,4,Screw,Red,14.0,London


SQLite supports two kinds of joins:
* Left join (`LEFT JOIN`): Keep all rows from the left table, merging rows from the right.
* Inner join (`INNER JOIN`): Keep only rows where there is a match.

Other DBMSes sometimes support two more kinds of joins:
* Right join: Keep all rows from the right table, merging rows from the left.
* Full join: Keep all rows from both tables, merging where there are matches.

In [49]:
pd.read_sql("SELECT * FROM supplierparts", db)

,PartID,SupplierID,Qty
0,1,1,300
1,1,2,200
2,1,3,400
3,1,4,200
4,1,5,100
5,1,6,100
6,2,1,300
7,2,2,400
8,3,2,200
9,4,2,200


In [50]:
pd.read_sql("""
    SELECT * FROM
    supplierparts AS l
    LEFT JOIN
    suppliers AS r
    ON l.supplierid = r.supplierid;
""", db)

,PartID,SupplierID,Qty,SupplierID,SupplierName,Status,City
0,1,1,300,1.0,Smith,20.0,London
1,1,2,200,2.0,Jones,10.0,Paris
2,1,3,400,3.0,Blake,30.0,Paris
3,1,4,200,4.0,Clark,20.0,London
4,1,5,100,5.0,Adams,30.0,Athens
5,1,6,100,NaN,None,NaN,None
6,2,1,300,1.0,Smith,20.0,London
7,2,2,400,2.0,Jones,10.0,Paris
8,3,2,200,2.0,Jones,10.0,Paris
9,4,2,200,2.0,Jones,10.0,Paris


In [51]:
pd.read_sql("""
    SELECT l.qty, r.* FROM
    supplierparts AS l
    LEFT JOIN
    suppliers AS r
    ON l.supplierid = r.supplierid;
""", db)

,Qty,SupplierID,SupplierName,Status,City
0,300,1.0,Smith,20.0,London
1,200,2.0,Jones,10.0,Paris
2,400,3.0,Blake,30.0,Paris
3,200,4.0,Clark,20.0,London
4,100,5.0,Adams,30.0,Athens
5,100,NaN,None,NaN,None
6,300,1.0,Smith,20.0,London
7,400,2.0,Jones,10.0,Paris
8,200,2.0,Jones,10.0,Paris
9,200,2.0,Jones,10.0,Paris


### Subqueries

You can write one or more `SELECT` queries within another:

In [52]:
pd.read_sql("""
    SELECT * FROM
        (SELECT l.*, r.supplierid, r.qty FROM
        parts AS l
        INNER JOIN
        supplierparts AS r
        ON l.partid = r.partid) AS a
    INNER JOIN
        (SELECT * FROM suppliers
        WHERE status > 10) AS b
    ON a.supplierid = b.supplierid;
""", db)

,PartID,PartName,Color,Weight,City,supplierid,qty,SupplierID,SupplierName,Status,City
0,1,Nut,Red,12.0,London,1,300,1,Smith,20,London
1,1,Nut,Red,12.0,London,3,400,3,Blake,30,Paris
2,1,Nut,Red,12.0,London,4,200,4,Clark,20,London
3,1,Nut,Red,12.0,London,5,100,5,Adams,30,Athens
4,2,Bolt,Green,17.0,Paris,1,300,1,Smith,20,London
5,4,Screw,Red,14.0,London,4,300,4,Clark,20,London
6,4,Screw,Red,14.0,London,5,400,5,Adams,30,Athens


You can paste strings into SQL queries, but make sure the string you paste in isn't compromised (e.g., user input)

![Drag Racing](https://imgs.xkcd.com/comics/exploits_of_a_mom_2x.png)

In [53]:
val = pd.read_sql("SELECT AVG(weight) FROM parts", db).iloc[0, 0]
val

15.166666666666666

In [56]:
query = "SELECT * FROM parts WHERE weight > " + format(val) #+ "; NEW QUERY"
query

'SELECT * FROM parts WHERE weight > 15.166666666666666'

In [57]:
pd.read_sql(query, db)

,PartID,PartName,Color,Weight,City
0,2,Bolt,Green,17.0,Paris
1,3,Screw,Blue,17.0,Oslo
2,6,Cog,Red,19.0,London


### Database Operations: CRUD

So far we've only learned 1/4 of the basic operations most databases support. They are:

* Create (`CREATE`): Create a new table.
* Read (`SELECT`): Read some rows from a table.
* Update (`UPDATE`): Update rows in a table.
* Delete (`DROP`): Delete a table.

For data analysis, you'll usually only need to read data.

Once in a while, it can be useful to create tables to store complicated results. We have to go back to __sqlite3__ to run SQL commands other than `SELECT`.

In [59]:
result = db.execute("""
    CREATE TABLE table2 AS
        SELECT l.*, r.qty, r.supplierid FROM
            parts AS l
        INNER JOIN
            supplierparts AS r
        ON l.partid = r.partid
""")

# Force the database to run ("commit") the query.
# Otherwise the database evaluates queries lazily, so the table will not show up.
db.commit()

OperationalError: table table2 already exists

In [60]:
pd.read_sql("SELECT * FROM table2", db)

,PartID,PartName,Color,Weight,City,Qty,SupplierID
0,1,Nut,Red,12.0,London,300,1
1,1,Nut,Red,12.0,London,200,2
2,1,Nut,Red,12.0,London,400,3
3,1,Nut,Red,12.0,London,200,4
4,1,Nut,Red,12.0,London,100,5
5,1,Nut,Red,12.0,London,100,6
6,2,Bolt,Green,17.0,Paris,300,1
7,2,Bolt,Green,17.0,Paris,400,2
8,3,Screw,Blue,17.0,Oslo,200,2
9,4,Screw,Red,14.0,London,200,2


In [61]:
pd.read_sql("SELECT * FROM sqlite_master", db)

,type,name,tbl_name,rootpage,sql
0,table,Suppliers,Suppliers,2,CREATE TABLE Suppliers (\n SupplierID integer...
1,table,Parts,Parts,3,"CREATE TABLE Parts (\n PartID integer,\n Par..."
2,table,SupplierParts,SupplierParts,4,CREATE TABLE SupplierParts (\n PartID integer...
3,index,sqlite_autoindex_SupplierParts_1,SupplierParts,5,None
4,table,table2,table2,6,"CREATE TABLE table2(\n PartID INT,\n PartNam..."


In [62]:
db.execute("DROP TABLE table2;")
db.commit()

In [63]:
db.close()

## Other Databases

You can use the __sqlalchemy__ package to connect to many different kinds of databases (not only SQLite).

The `sqlalchemy.create_engine()` function creates an _engine_ to connect to a database. The `.raw_connection()` method on the returned engine opens a connection compatible with Pandas.

See the __sqlalchemy__ documentation for more info about [how to write a database URL](https://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls).

In [ ]:
import sqlalchemy

engine = sqlalchemy.create_engine("sqlite:///../data/suppliers.sqlite")
conn = engine.raw_connection()

In [ ]:
conn

In [ ]:
pd.read_sql("SELECT * FROM sqlite_master", conn)

In [ ]:
engine.table_names()

In [ ]:
conn.close()

Unlike SQLite, most databases:

* Are not stored in a single file.
* Have a client-server design, where you connect to the database by logging in with a username and password.